# Integrate an External Test Provider

This notebook demonstrates how to use a custom test provider to be able to use custom tests with the Validmind Developer Framework.
In the notebook, we load a couple different demo test providers and register them with the Validmind framework to be able to run a template that utilizes those tests.

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
%pip install --upgrade validmind

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [9]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    api_secret="API_SECRET",
    project="xxxxxxxxxxxxxxxxxxxxxxxxx"
)

2023-07-10 13:58:18,343 - INFO - api_client - Connected to ValidMind. Project: Customer Churn Model - Test for Notebooks, Sydney (cljxccag7000016mkriy2l77l)


##### Update your template

Within the project you would like to use, create a new Content Block in the documentation template and add the following code:

```yaml
      - content_type: metric
        content_id: my_local_provider.tests.MyCustomTest
      - content_type: metric
        content_id: my_inline_provider.tests.MyCustomTest
```

See the section below on setting up and registering test providers for more info on how these content_id's get mapped to the actual test providers and tests.

### Preview the template for the current project to validate that it looks correct

In [3]:
# we should see two custom content blocks in the template whose IDs are under the namespaces registered below (`my_inline_provider` and `my_local_provider`)
# the ID should match the path to the `tests` directory in this repo
vm.preview_template()

Accordion(children=(Accordion(children=(HTML(value='<p>Empty Section</p>'), Accordion(children=(HTML(value='<p…

### Register external test providers

We will now instantiate and register the Test Provider classes that we imported above.

For the Github provider, we will need to specify the `org` and `repo` to pull from as well as optionally pass a `token` if the repo is private.

For the Local Filesystem provider, we will just need to specify the root folder under which the provider class will look for tests. In this case, it is the current directory for this demo so you may have to adjust the path for your machine.

**Import the Local File System Test Provider from the `validmind.tests` module**

In [4]:
from validmind.tests import LocalTestProvider

**Create an inline TestProvider Class that just returns a single test**

In [6]:
import pandas as pd

class MySecondCustomTest(vm.vm_models.Metric):
    # The metric name should match the content ID on the template
    name = "my_inline_provider.tests.MyCustomTest"

    def description(self):
        return "This is a custom test from an external test provider."

    def run(self):
        return self.cache_results([{"foo": "bar"}])

    def summary(self, results):
        return vm.vm_models.ResultSummary(
            results=[
                vm.vm_models.ResultTable(
                    data=pd.DataFrame(results),
                    metadata=vm.vm_models.ResultTableMetadata(
                        title="Results from Test Provider Inside Notebook"
                    ),
                )
            ]
        )


class TestProviderInline:
    def load_test(self, test_id):
        # ignore the test_id and just return the single test above
        return MySecondCustomTest

In [7]:
# instantiate the test provider
inline_test_provider = TestProviderInline()
local_test_provider = LocalTestProvider(root_folder=".")

# register the test providers
vm.tests.register_test_provider(
    namespace="my_inline_provider",
    test_provider=inline_test_provider,
) # validmind will now call the `TestProviderInline.load_test` method whenever it encounters a test ID that starts with `my_inline_provider`

vm.tests.register_test_provider(
    namespace="my_local_provider",
    test_provider=local_test_provider,
) # validmind will now call the `LocalTestProvider.load_test` method whenever it encounters a test ID that starts with `my_local_provider`

### Running the template

Now we can run the template as usual and it will use the external test providers to load the appropriate tests. Note that we're not passing any context such as `dataset` and `model` to `run_template()`. This is because the template we're using for this demo only has a single section and does not require any additional context to run.

In [8]:
vm.run_template()

KeyError: 'mean_of_values'